### Collective Burden for Sequencing Documents

In [105]:
import requests
from collections import Counter
import random as randomlib

from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import networkx as nx

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [106]:
'''
Knowledge Graph
'''
kg_path = "../graph_query/graphs/knowledge_graph.gpickle"
kg = nx.read_gpickle(kg_path)
kg_labels = [str(x) for x in list(kg.nodes())[1:]]
n_labels = len(kg_labels)

In [107]:
def get_queries_based_on_node(node_label):
    node = kg.node[node_label]

    if("NodeType" not in node): 
        kg.node[node_label]["NodeType"] = "ConceptNode"
        l =  list(kg.neighbors(node_label))
        return l
    node_type = node["NodeType"]
    if(node_type == "TopicNode" or node_type == "ConceptNode"):
        return list(kg.neighbors(node_label))
    elif(node_type == "SubConceptNode"):
        return [node_label]
    else:
        pass

'''
Returns a list of queries depending on the type of the
node closest to the query.
args - query(str)
returns [] of str
'''
def query_formulator(query, label):
    queries = []
    children_neighbours = get_queries_based_on_node(label)
    queries = [label]
    for child in children_neighbours:
        queries.append(child)
    return list(set(queries))


In [108]:
'''
Get content from a given set of URLs.
'''
es_order = []
def get_content():
    f = open('lps/user_study_dynamic_programming_engage.txt', 'r')
    l = f.readlines()
    docs = {}
    index = {}
    
    counter = 0
    for url in l:
        try:
            docs[url] = requests.get(url).content
            index[url] = counter
            es_order.append(url)
            counter += 1
        except:
            continue
    return docs, index

In [109]:
'''
Term Frequency Array for a particular document.
'''
def get_tfd(content):
    word_count_dict = Counter(w for w in kg_labels 
                              if w.lower() in content.lower())
    common = word_count_dict.most_common()
    
    frequency_arr = [0]*len(kg_labels)
    
    for common_word in common:
        common_word_index = kg_labels.index(common_word[0])
        frequency_arr[common_word_index] = common_word[1]
    return frequency_arr

In [110]:
content, index = get_content()

In [111]:
'''
Building word_data a document (rows) by term frequency (columns) matrix.
'''
tfd_data = {}
for url, cont in content.items():
    tfd_data[url] = get_tfd(cont)

tfd_arr = []
for key in index.keys():
    tfd_arr.append(key.replace("\n", ""))

word_data = {'TFD':tfd_arr}

for label in kg_labels:
    word_data[label] = [None]*len(index)

for url, words_in_doc in tfd_data.items():
    url_index = index[url]
    for i in range(0, n_labels, 1):
        word = kg_labels[i]
        word_data[word][url_index] = words_in_doc[i]

In [112]:
'''
(DTF)^T(DTF) = Coocurence Matrix
'''
document_term_frequency = pd.DataFrame(word_data).set_index('TFD')
dtf_asint = document_term_frequency.astype(int)
coocc = dtf_asint.T.dot(dtf_asint)

### Calculating Relationship Score: S(i, j)

In [113]:
def get_relationship_between_concepts(concept_1, concept_2):
    concept_1_index= document_term_frequency.columns.get_loc(concept_1)
    concept_2_index= document_term_frequency.columns.get_loc(concept_2)
    
    return coocc.iloc[concept_1_index, concept_2_index]

### Significance of a concept in a document: \lambda(c, i)

In [114]:
def get_significance_score(concept, document):
    if(document == None): return 0
    concept_index = document_term_frequency.columns.get_loc(concept)
    freq = dtf_asint.iloc[index[document]][concept_index]
    coocc_row = coocc.iloc[concept_index,:] 
    r = np.array(coocc_row)
    if(sum(r) == 0): return 0
    return (freq)*np.count_nonzero(r)

### Key Sections k_c

In [115]:
def get_doc_to_concepts_list(content, top_n):
    doc_to_concept_list = {}
    
    for each_document in content.keys():
        rt = []
        rc = []
        for each_concept in kg_labels:
            s = get_significance_score(each_concept, each_document)
            if("NodeType" not in kg.node[each_concept]):
                kg.node[each_concept]["NodeType"] = "ConceptNode"
                rc.append((each_concept, s))
            elif(kg.node[each_concept]["NodeType"] == "ConceptNode"):
                rc.append((each_concept, s))
            elif(kg.node[each_concept]["NodeType"] == "TopicNode"):
                rt.append((each_concept, s))
        
        rt.sort(key=lambda x:x[1])
        rt = rt[::-1]
        
        rc.sort(key=lambda x:x[1])
        rc = rc[::-1]
        
        key_concepts = []
        
        while(len(key_concepts) < top_n and len(rc)):
            key_concepts.append(rc[0][0])
            rc.pop(0)
            
        while(len(key_concepts) < top_n and len(rt)):
            key_concepts.append(rt[0][0])
            rt.pop(0)
        
        doc_to_concept_list[each_document] = key_concepts
    return doc_to_concept_list

In [116]:
def get_relevant_concepts_for_lp(doc_to_concepts_list):
    rel = []
    for key,val in doc_to_concepts_list.items():
        for each in val:
            rel.append(each)
    return rel   

### Comprehension Burden

In [139]:
def f_cb(sig_score, key_sig_score, relationship):
    return key_sig_score

def get_cb_document(document, document_key_concepts, visited, relevant):
    document_burden = 0.0
    for other_concept in relevant:
        if(other_concept in visited or other_concept in document_key_concepts): continue
        
        other_concept_burden = 0
        for each_key_concept in document_key_concepts:
            sig_score = get_significance_score(other_concept, document)
            key_sig_score = get_significance_score(each_key_concept, document)
            relationship = get_relationship_between_concepts(each_key_concept, 
                                                             other_concept) 
            other_concept_burden += f_cb(sig_score, key_sig_score, relationship)
        other_concept_burden = other_concept_burden/len(document_key_concepts)
        document_burden += other_concept_burden        
    return document_burden

### Sequence Generation

In [140]:
def get_linear(nodes):
    parents = []
    for each in nodes:
        if(kg.nodes[each]["NodeType"] == "TopicNode"):
            parents.append(each)
    
    linear = []
    for p in parents:
        linear.append(p)
        children = kg.neighbors(p)
        for c in children:
            if c in nodes and kg.nodes[c]["NodeType"] == "ConceptNode":
                linear.append(c)
    
    for each in nodes:
        if each not in linear:
            linear.append(each)
    
    return linear

def get_es(nodes, key_doc):
    order =[]
    for each in es_order:
        each = each.replace("\n", "")
        
        if(each in key_doc and key_doc[each] not in order):
            order.append(key_doc[each])
    for each in nodes:
        if (each not in order):
            order.append(each)
    return order
    
def get_sequences(nodes, key_doc):
        random = get_es(nodes, key_doc)
        linear = get_linear(nodes)
        top_down = linear[::-1]
        return random, linear, top_down

In [141]:
def get_concepts_to_document_list(doc_to_concepts_list, relevant_concepts):
    concept_to_document_list = {}
    for each_concept in relevant_concepts:
        concept_to_document_list[each_concept] = []
        for doc, kcs in doc_to_concepts_list.items():
            if(each_concept in kcs): 
                concept_to_document_list[each_concept].append(doc)
    return concept_to_document_list
                

In [142]:
def get_burden_for_sequence(sequence, doc_to_concepts_list, concepts_to_document_list):
    collective_burden = 0.0
    
    visited = set()
    for each_relevant_concept in sequence:
        visited.add(each_relevant_concept)
        docs_assoc = concepts_to_document_list[each_relevant_concept]
        for d in docs_assoc:
            for other_keys in doc_to_concepts_list[d]:
                visited.add(other_keys)
            collective_burden += get_cb_document(d, doc_to_concepts_list[d], visited, sequence)
    return collective_burden    

In [147]:
doc_to_concepts_list = get_doc_to_concepts_list(content, 1)
relevant_concepts_to_sequence = list(set(get_relevant_concepts_for_lp(doc_to_concepts_list)))
concepts_to_document_list = get_concepts_to_document_list(doc_to_concepts_list, relevant_concepts_to_sequence)

es, linear, bottom_up = get_sequences(relevant_concepts_to_sequence, doc_to_concepts_list)

print(get_burden_for_sequence(linear, doc_to_concepts_list, concepts_to_document_list)/len(doc_to_concepts_list))

# get_burden_for_sequence(bottom_up, doc_to_concepts_list, concepts_to_document_list)

81.2142857143
